<a href="https://colab.research.google.com/github/joojinho97/2022_GC_Covid19_prediction/blob/main/multi_input_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install medpy

     |████████████████████████████████| 151 kB 14.2 MB/s 
     |████████████████████████████████| 48.4 MB 1.2 MB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754460 sha256=f1b75dc156a7cab41587bc79d9a9c1c856a8f74c400ab3012caa71879e9e5eb0
  Stored in directory: /root/.cache/pip/wheels/b0/57/3a/da1183f22a6afb42e11138daa6a759de233fd977a984333602
Successfully built medpy


In [ ]:
import medpy
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import pandas as pd
import numpy as np
import cv2
import SimpleITK as sitk
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense,Flatten,Conv2D,Concatenate,MaxPooling2D,BatchNormalization,Activation,Add,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB7,EfficientNetB6,EfficientNetB5,EfficientNetB3,EfficientNetB1,ResNet101,ResNet50,ResNet152

In [ ]:
import pathlib

In [ ]:
from medpy.io import load

In [ ]:


os.chdir('/content/drive/MyDrive/grand_challenge/destination/data/mha/')

data_list_f=glob.glob('*.mha')
data_list_f.sort(key=lambda fname: int(fname.split('.')[0]))



data_list_3=glob.glob('*.mha')
data_list_3.sort(key=lambda fname: int(fname.split('.')[0]))

data_list_4=glob.glob('*.mha')
data_list_4.sort(key=lambda fname: int(fname.split('.')[0]))

for i in range(len(data_list_3)):
  
  
  data_list_3[i]='/content/drive/MyDrive/grand_challenge/destination/data/test_0115/'+data_list_3[i]
  data_list_4[i]='/content/drive/MyDrive/grand_challenge/destination/data/test_0111/'+data_list_4[i]
  data_list_f[i]='/content/drive/MyDrive/grand_challenge/destination/data/mha/'+data_list_f[i]

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/grand_challenge/destination/metadata/reference.csv')
data=data.sort_values(by='PatientID')


In [ ]:
data_ag=pd.read_csv('/content/drive/MyDrive/grand_challenge/destination/metadata/age_gender.csv')
data_ag=list(zip(data_ag['age'].tolist(),data_ag['gender'].tolist()))


In [ ]:
train_data_ag=data_ag[:1400]
val_data_ag=data_ag[1401:]

In [ ]:
tr=list(zip(data["probCOVID"].tolist(), data["probSevere"].tolist()))

In [ ]:
train_list_f=data_list_f[:1400]
val_list_f=data_list_f[1401:]


In [ ]:
train_list_4=data_list_4[:1400]
val_list_4=data_list_4[1401:]

In [ ]:
train_list_3=data_list_3[:1400]
val_list_3=data_list_3[1401:]
train_label=tr[:1400]
val_label=tr[1401:]

In [ ]:
def pre(img,img4):
  
  img  = cv2.normalize(np.float32(img),None,img.numpy().min(),img.numpy().max(),cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  img4  = cv2.normalize(np.float32(img4),None,img4.numpy().min(),img4.numpy().max(),cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  if int(len(img.shape)) == 2:
    img=np.expand_dims(img,axis=-1)
  if int(len(img4.shape)) == 2:
    img4=np.expand_dims(img4,axis=-1)
  return img,img4
  
def data_(dataset):
  dataset=dataset.map(file_replace)
  dataset=dataset.map(data_preprocess)  
  return  dataset

def file_replace(path,path4,data_ag,labels):

  return tf.strings.regex_replace(path, '.mha', ''), tf.strings.regex_replace(path4, '.mha', '.jpg'),data_ag,labels

def data_preprocess(x,path4, data_ag,labels):
  
  raw=tf.io.read_file(x)
  img = tf.image.decode_png(raw, channels=4)

  raw4=tf.io.read_file(path4)
  img4 = tf.image.decode_png(raw4, channels=3)
  
  img,img4=tf.py_function(pre,inp=[img,img4],Tout=(tf.float32,tf.float32))
  data_ag=tf.cast(data_ag,tf.int8)
  
  return {'input_1':img,'input_2':img4, 'age':data_ag[0],'gender':data_ag[1]}, {'probCOVID':labels[0], 'probSevere' :labels[1]}
  

In [ ]:
def make_dataset(train_list_3,train_list_4,data_ag, tr):
  test_flist=tf.data.Dataset.from_tensor_slices((train_list_3,train_list_4,data_ag,tr))
  test_dataset=data_(test_flist)
  test_dataset=test_dataset.batch(2)
  test_dataset=test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
  test_datset = test_dataset.apply(tf.data.experimental.ignore_errors())
  return test_dataset


In [ ]:
train_dataset=make_dataset(train_list_3,train_list_4, train_data_ag,train_label)
val_dataset=make_dataset(val_list_3,val_list_4,val_data_ag,val_label)

In [ ]:
for i,j in train_dataset.take(1):
  print(i)

In [ ]:
input_img=Input(shape=(512,512,4),name='input_1')
input_shape=(512,512,4)
input_img2=Input(shape=(256,256,3),name='input_2')
input_shape2=(256,256,3)
input_age=Input(shape=(1,),name='age')
input_gender=Input(shape=(1,),name='gender')

In [ ]:
ef=ResNet50(weights=None,include_top=False, input_shape=input_shape, pooling="max")(input_img)
ef2=ResNet101(weights=None,include_top=False, input_shape=input_shape2, pooling="max")(input_img2)



age_dense = tf.keras.layers.Dense(1, activation='relu')(input_age)
gender_dense = tf.keras.layers.Dense(1, activation='relu')(input_gender)

covid_dense=Concatenate()([ef,ef2])
covid_dense=BatchNormalization()(covid_dense)

predictions_covid = tf.keras.layers.Dense(1, activation='sigmoid',name='probCOVID')(covid_dense)

severe_dense=Concatenate()([ef,ef2,input_gender,input_age,predictions_covid])
severe_dense=BatchNormalization()(severe_dense)
#*5 빼고해보기
#ef ef2 빼고 3개에다가 conv1d하나써서 해보는 방식

predictions_severe = tf.keras.layers.Dense(1, activation='sigmoid',name='probSevere')(severe_dense)
model = tf.keras.Model(inputs=[input_img,input_img2,input_age,input_gender], outputs=[predictions_covid,predictions_severe])

/usr/local/lib/python3.7/dist-packages/keras/applications/resnet.py:141: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  weights=weights)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss={'probCOVID': 'binary_crossentropy', 'probSevere':'binary_crossentropy'},metrics=[tf.keras.metrics.AUC(multi_label=True, curve='ROC', name='auc_pr')])
# loss_weight=[1,2] 이런식으로 가중치 부여도 해보기 ,loss_weights=[1,7]

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/grand_challenge/destination/data/h5/resnet50/{epoch:02d}_A.h5', verbose=True, save_weights_only=True, mode='auto'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_probCOVID_auc_pr', factor=0.8, patience=2, verbose=0, mode='max', min_delta=0.0001, cooldown=0, min_lr=0),
]

In [ ]:
model.fit(train_dataset, epochs=150,validation_data=val_dataset,callbacks=callbacks)

In [ ]:
input_image = sitk.ReadImage('/content/drive/MyDrive/grand_challenge/destination/data/mha/6.mha')

베스트 결과 모델 구조 +  val 결과


In [ ]:

for i in range(len(data_list_3)):
  
  
  data_list_3[i]='/content/drive/MyDrive/grand_challenge/destination/data/test_0115/'+data_list_3[i]
  data_list_4[i]='/content/drive/MyDrive/grand_challenge/destination/data/test_0111/'+data_list_4[i]
  data_list_f[i]='/content/drive/MyDrive/grand_challenge/destination/data/mha/'+data_list_f[i]

In [ ]:
def pre(img,img4):
  img  = cv2.normalize(np.float32(img),None,0,1,cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  img4  = cv2.normalize(np.float32(img4),None,0,1,cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  if int(len(img.shape)) == 2:
    img=np.expand_dims(img,axis=-1)
  if int(len(img4.shape)) == 2:
    img4=np.expand_dims(img4,axis=-1)
  return img,img4

def data_(dataset):
  dataset=dataset.map(file_replace)
  dataset=dataset.map(data_preprocess)  
  return  dataset

def file_replace(path,path4,data_ag,labels):

  return tf.strings.regex_replace(path, '.mha', ''), tf.strings.regex_replace(path4, '.mha', '.jpg'),data_ag,labels

def data_preprocess(x,path4, data_ag,labels):
  
  raw=tf.io.read_file(x)
  img = tf.image.decode_png(raw, channels=4)

  raw4=tf.io.read_file(path4)
  img4 = tf.image.decode_png(raw4, channels=3)
  
  img,img4=tf.py_function(pre,inp=[img,img4],Tout=(tf.float32,tf.float32))
  data_ag=tf.cast(data_ag,tf.int8)
  
  return {'input_1':img,'input_2':img4, 'age':data_ag[0],'gender':data_ag[1]}, {'probCOVID':labels[0], 'probSevere' :labels[1]}
  

In [ ]:
input_img=Input(shape=(512,512,4),name='input_1')
input_shape=(512,512,4)
input_img2=Input(shape=(512,512,3),name='input_2')
input_shape2=(512,512,3)
input_age=Input(shape=(1,),name='age')
input_gender=Input(shape=(1,),name='gender')

In [ ]:
ef=ResNet50(weights=None,include_top=False, input_shape=input_shape, pooling="max")(input_img)
ef2=ResNet101(weights=None,include_top=False, input_shape=input_shape2, pooling="max")(input_img2)

age_dense = tf.keras.layers.Dense(1, activation='relu')(input_age)
gender_dense = tf.keras.layers.Dense(1, activation='relu')(input_gender)

covid_dense=Concatenate()([ef,ef2])
predictions_covid = tf.keras.layers.Dense(1, activation='sigmoid',name='probCOVID')(covid_dense)

severe_dense=Concatenate()([ef,ef2,input_gender,input_age,predictions_covid])
predictions_severe = tf.keras.layers.Dense(1, activation='sigmoid',name='probSevere')(severe_dense)


model = tf.keras.Model(inputs=[input_img,input_img2,input_age,input_gender], outputs=[predictions_covid,predictions_severe])

In [ ]:
val_probCOVID_auc_pr: 0.6844 - val_probSevere_auc_pr: 0.7562 => 0.7203 epoch21